# Stochastic Block Model (SBM)

In [ ]:
import graspologic

import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

Unlike [Erdos-Renyi (ER) models](./erdos_renyi.ipynb), a Stochastic Block Model (SBM) produces graphs containing communities: disjoint subgraphs characterized by differing edge probabilities for vertices within and between communities<sup>[1](https://en.wikipedia.org/wiki/Stochastic_block_model)</sup>.

SBM is parametrized by the number of vertices in each community $n$, and a block probability matrix $P \in \mathbb{R}^{n x n}$ where each element specifies the probability of an edge in a particular block. One can think of SBM as a collection of ER graphs where each block corresponds to an ER graph.

Below, we sample a two-block SBM (undirected, no self-loops) with following parameters:

\begin{align*}
n &= [50, 50]\\
P &= \begin{bmatrix} 
0.5 & 0.2\\
0.2 & 0.05
\end{bmatrix}
\end{align*}

The diagonals correspond to probability of an edge within blocks and the off-diagonals correspond to probability of an edge between blocks.

In [ ]:
from graspologic.simulations import sbm

n = [50, 50]
p = [[0.5, 0.2],
     [0.2, 0.05]]

np.random.seed(1)
G = sbm(n=n, p=p)

## Visualize the graph using heatmap

In [ ]:
from graspologic.plot import heatmap

heatmap(G, title ='SBM Simulation')

## Weighted SBM Graphs

Similar to ER simulations, *sbm* functions provide ways to sample weights for all edges that were sampled via a probability distribution function. In order to sample with weights, you can either:

1. Provide a *single* probability distribution function with corresponding keyword arguments for the distribution function. All weights will be sampled using the same function.
2. Provide a probability distribution function with corresponding keyword arguments for each block.

Below we sample a SBM (undirected, no self-loops) with the following parameters:

\begin{align*}
n &= [50, 50]\\
P &= \begin{bmatrix}0.5 & 0.2\\
0.2 & 0.05
\end{bmatrix}
\end{align*}

and the weights are sampled from the following probability functions:

\begin{align*}
PDFs &= \begin{bmatrix}Normal & Poisson\\
Poisson & Normal
\end{bmatrix}\\
Parameters   &= \begin{bmatrix}{\mu=3, \sigma^2=1} & {\lambda=5}\\
{\lambda=5} & {\mu=3, \sigma^2=1}
\end{bmatrix}
\end{align*}

In [ ]:
from numpy.random import normal, poisson

n = [50, 50]
p = [[0.5, 0.2],
     [0.2, 0.05]]
wt = [[normal, poisson],
      [poisson, normal]]
wtargs = [[dict(loc=3, scale=1), dict(lam=5)],
          [dict(lam=5), dict(loc=3, scale=1)]]

G = sbm(n=n, p=p, wt=wt, wtargs=wtargs)

## Visualize the graph using heatmap

In [ ]:
heatmap(G, title='Weighted SBM Simulation')

# Mixed Membership Stochastic Blockmodel(MMSBM)

In [1]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

Given a network with $k$ number of communities, a SBM is parametrized by the number of vertices in each community $n$; a block connectivity matrix	$B\in R^{kxk}$ where each $(i^{th}, j^{th})$ entry specifies the probability of connection between nodes pertaining to the $i^{th}$ community and the $j^{th}$ community; and a membership matrix $Z\in R^{nxk}$ in which the $i^{th}$ row vector contains information regarding which of the $k$ communities the node $i$ pertains to. In fact, for any $i^{th}$ row of $Z$ each $j^{th}$ entry indicates the probability of the node $i$ pertaining to the community $j$. Each edge between nodes is then sampled from Bernoulli distribution according to a probability matrix $P\in R^{nxn}$ such that $P$ = $Z B Z^{T}$ and each $(i^{th}, j^{th})$ entry indicates the probability of connection between node $i$ and $j$.

One drawback of utilizing SBM is that each node must be assigned exclusively to a single community. Therefore, each row of $Z$ contains $1$ in the $j^{th}$ column indicating the community to which that node belongs to and $0$ for all the other entries. 

The Mixed Membership Stochastic Block Model (MMSBM) addresses this issue by adding some variability in the community assignment process to each node. Indeed, given a network with $k$ communities and a connectivity matrix $B\in R^{kxk}$ each node $p$ is assigned a membership vector $\vec{\pi}_p$. As aforementioned, the $i^{th}$ entry of this vector indicates the probability of node $p$ pertaining to the $i^{th}$ community when interacting with another node. Differently from SBM, a node is allowed to pertain to multiple communities. This is achieved by estimating how often a node acts as belonging to a community with respect to another when forming connections with other nodes. For example consider:

\begin{align*}
\vec{\pi}_p &= \begin{bmatrix} 1 & 0 & 0 & 0 \end{bmatrix}\\
\vec{\pi}_q &= \begin{bmatrix} 0.25 & 0.25 & 0.25 & 0.25 \end{bmatrix}\\
\end{align*}

According to $\vec{\pi}_p$, node $p$ will always behave as pertaining to the first community when interacting with other nodes, while node $q$ will interact 25% of the time as if belonging to community 1, 25% of the time as if belonging to community 2, etc.

In MMSBM, this membership assignment is performed by sampling each membership vector $\vec{\pi}_p$ from a Dirichlet distribution with concentration parameter $\alpha$ so that by modifying $\alpha$ we can control the shape of the Dirichlet distribution and thus indirectly the assigned membership vectors. For example, when one of the entries in $\alpha$ is much greater than the others, the majority of the nodes will pertain to the community associated with that entry. If all entries of $\alpha$ are identical to each other then values of the concentration parameter closer to $0$ will cause most of the nodes to belong in only one community similarly to an SBM. Always maintaining all entries identical, as we increase the values in $\alpha$ more nodes are allowed to pertain to multiple communities, therefore the underlying block structure is progressively lost.

To show this we sample a four-block SBM (undirected, no self-loops) graph with following parameters:
\begin{align*}
k &= 4\\
n &= \begin{bmatrix} 50 & 50 & 50 & 50 \end{bmatrix}\\
B &=\begin{bmatrix} 0.8 & 0.2 & 0.2 & 0.2 \\ 0.2 & 0.8 & 0.2 & 0.2 \\ 0.2 & 0.2 & 0.8 & 0.2 \\ 0.2 & 0.2 & 0.2 & 0.8 \end{bmatrix}
\end{align*}

Similarly, we also sample from MMSBM a graph with the same number of communities, same connectivity matrix and same total number of vertices:
\begin{align*}
n &=200\\
k &= 4\\
\alpha &= \begin{bmatrix} 0.05 & 0.05 & 0.05 & 0.05 \end{bmatrix}\\
B &=\begin{bmatrix} 0.8 & 0.2 & 0.2 & 0.2 \\ 0.2 & 0.8 & 0.2 & 0.2 \\ 0.2 & 0.2 & 0.8 & 0.2 \\ 0.2 & 0.2 & 0.2 & 0.8 \end{bmatrix}
\end{align*}

Note that, differently from SBM, for the MMSBM we do not specify the number of vertices pertaining to each specific block as the assignment is performed via random sampling from Dirichlet distribution.


In [ ]:
from graspologic.simulations import sbm, mmsbm
seed = 1

rng = np.random.default_rng(seed)
np.random.seed(seed)

n = [50] * 4
p = [[0.8,0.2,0.2,0.2],
    [ 0.2,0.8,0.2,0.2],
    [ 0.2,0.2,0.8,0.2],
    [ 0.2,0.2,0.2,0.8]]

G_sbm = sbm(n, p, directed=False, loops=False)

n= 200
k = 4
alpha = [0.05]*k

G_mmsbm = mmsbm(n, p, alpha= alpha, rng = rng)

## Plot SBM and MMSBM graphs using heatmap

In [ ]:
from graspologic.plot import heatmap

heatmap(G_sbm, title ='SBM Simulation');
heatmap(G_mmsbm, title ='MMSBM Simulation(alpha = 0.05)');

As expected, the two graphs appear to be generated from four-block SBMs with connectivity matrix $B$. In the MMSBM case, the number of nodes assigned to each block is randomly sampled from the Dirichlet distribution compared to the SBM in which it is specified, which is why we can see different numbers of nodes per group.

Now let's try increasing the values of each entry of the concentration parameter $\alpha$ first to $0.25$, then $0.5$ and finally to $1$.

In [ ]:
alpha = [0.25]*k
print("First increase alpha to: " + str(alpha))

G_mmsbm_25 = mmsbm(n, p, alpha= alpha, rng = rng)
heatmap(G_mmsbm_25, title ='MMSBM Simulation(alpha = 0.25)');

alpha = [0.5]*k
print("Then increase alpha to: " + str(alpha))

G_mmsbm_50 = mmsbm(n, p, alpha= alpha, rng = rng)
heatmap(G_mmsbm_50, title ='MMSBM Simulation(alpha = 0.5)');

alpha = [1.0]*k
print("Finally increase alpha to: " + str(alpha))

G_mmsbm_100 = mmsbm(n, p, alpha= alpha, rng = rng)
heatmap(G_mmsbm_100, title ='MMSBM Simulation(alpha = 1.0)');

As expected, as we increase the value of alpha we increase the mixed membership allowed at each node so that the community structure is progressively lost.